In [1]:
# Load libraries for file handling and image crunching
import numpy as np
import matplotlib.pyplot as plt
import cupy as cp

import scipy.ndimage as ndicpu
import cupyx.scipy.ndimage as ndi

import seaborn as sns
import pandas as pd
# Set matplotlib backend
%matplotlib inline 

import cucim.skimage as skimage
import skimage as skimagecpu
# Import the os module
import os

#fancy gui viewer
import napari

# progress bar for long computation
from tqdm import tqdm

# import own helper functions to subset and make boxes from coordinates
from boxhelpers_cp import *

In [2]:
wdpath = os.getcwd()

from MGlia_detect_utils import *

# grab a testimage
testimage = "C2-220421 otof_iba slide002 mouse195 005.tif"

testimagebrain = "320763 CNIC.tif"
dirpath = "./"

filepath = os.path.join(wdpath, testimage)

# load the image
from skimage.io import imread
img = imread(filepath)

In [3]:
# Setup the dimension of the image
planestep = 0.3
xystep = 0.27500004812500844
pixvol =planestep*xystep**2


In [4]:
#convert img to cp array on GPU
#gpu_img = cp.asarray(img)

# monitor memory of the GPU
mempool = cp.get_default_memory_pool()

def getGPUmem():
    #calculate the proportion of memory used an return
    used = mempool.used_bytes()/mempool.total_bytes()
    return used


In [5]:
# run a gaussian filter on GPU
filtered = ndi.filters.gaussian_filter(cp.asarray(img), 5).get()

In [6]:
# define a cube of x microns as a footprint
# use the scale and floor division to find the number of pixels in each dimension to use
x = 30
foot = cp.ones((int(x//planestep)//3, # use a smaller z step to not reject more candidate seeds (purely empirical)
               int(x//xystep),
               int(x//xystep)))


In [7]:
locmax = ndi.maximum_filter(cp.array(filtered), footprint = foot).get()

candmask = locmax  == filtered

#dilate the seed for plotting
candmask_plot = ndi.binary_dilation(cp.array(candmask), structure = cp.ones((3,10,10))).get()

In [8]:
viewer = napari.view_image(img)
#new_layer = viewer.add_image(cp.asnumpy(gpu_locmax), opacity = 0.2, colormap = "red")
new_layer = viewer.add_image(candmask_plot, opacity = 0.5, colormap = "red")


In [9]:
# with skimage maximum filter (returns coordinates, not pixels)
locmax = skimage.feature.peak_local_max(cp.array(filtered), min_distance=0, footprint = foot).get()

#create an empty boolean array of the dimensions of the source img
localhigh = np.zeros_like(filtered, dtype=bool)

# this will feed the coord to the empty mask
localhigh[tuple(locmax.T)] = True

In [10]:
# label the local highs and inspect the seeds generated
localhigh_img = ndi.label(cp.array(localhigh))[0].get()

localhigh_img_plot = ndi.binary_dilation(cp.array(localhigh_img), structure = cp.ones((3,10,10))).get()

In [11]:
viewer = napari.view_image(img)
new_layer = viewer.add_image(localhigh_img_plot, opacity = 0.2, colormap = "red")
new_layer = viewer.add_image(localhigh_img, opacity = 0.2, colormap = "red")


In [12]:
# make a helper function to span a box around a 3d pixel coordinate
def seed_to_box(image, coords, npixels):
    # subset the box and set pixels to ones
    
    # the desired box gets spanned in 2 directions, we need to half this
    npixels = npixels//2
    # image boundaries
    boundaries = image.shape
    
    #print(boundaries)
    zstart = coords[0] - npixels
    zstop  = coords[0] + npixels
    
    xstart = coords[1] - npixels
    xstop  = coords[1] + npixels
    
    ystart = coords[2] -npixels
    ystop  = coords[2] + npixels
    # set fallback if image borders are touched
    if zstart < 0:
        zstart = 0
        
    if xstart < 0:
        xstart = 0
    
    if ystart < 0:
        ystart = 0
    
    # set fallback for end being larger than image boundaries
    if zstop > boundaries[0]:
        zstop = boundaries[0]
    
    if xstop > boundaries[1]:
        xstop = boundaries[1]
    
    if ystop > boundaries[2]:
        ystop = boundaries[2]
        
    box = cp.zeros_like(image)
    # switch on pixels in the box
    box[zstart:zstop,xstart:xstop, ystart:ystop] = True 
    # push to mem and return
    return np.array(box.get()).astype(bool)

In [13]:
def seed_to_subset(image, coords, npixels):
        
    # the desired box gets spanned in 2 directions, we need to half this
    npixels = npixels//2
    # image boundaries
    boundaries = image.shape
    
    #print(boundaries)
    zstart = coords[0] - npixels
    zstop  = coords[0] + npixels
    
    xstart = coords[1] - npixels
    xstop  = coords[1] + npixels
    
    ystart = coords[2] -npixels
    ystop  = coords[2] + npixels
    # set fallback if image borders are touched
    if zstart < 0:
        zstart = 0
        
    if xstart < 0:
        xstart = 0
    
    if ystart < 0:
        ystart = 0
    
    # set fallback for end being larger than image boundaries
    if zstop > boundaries[0]:
        zstop = boundaries[0]
    
    if xstop > boundaries[1]:
        xstop = boundaries[1]
    
    if ystop > boundaries[2]:
        ystop = boundaries[2]
        
    # subset the image + return
    imgbox = image[zstart:zstop,xstart:xstop, ystart:ystop]
    return imgbox

In [14]:
subset = seed_to_subset(img, locmax[100], int(50//xystep))
viewer = napari.view_image(subset)


In [15]:
# test the functions
viewer = napari.view_image(img)

seed20 = seed_to_box(img, locmax[20] , 2//xystep)
seed20box = seed_to_box(img, locmax[20] , 100//xystep)

new_layer = viewer.add_image(seed20, opacity = 0.2, colormap = "red")
new_layer = viewer.add_image(seed20box, opacity = 0.2, colormap = "cyan")

In [16]:
# define a function that makes a local threshold
def find_cell_thresh(image, seed, expandpix):
    # subset the ROI and calulate thresh based on ROI
    ROI = seed_to_subset(image, seed, expandpix)
    Thresh = skimagecpu.filters.threshold_otsu(ROI)
    return Thresh

In [17]:
# define a function that fills a cell from a seed given a threshold
# unfortunately to date floodfill is not yet implemented on cupy or cucim
# use CPU
import skimage.segmentation as CPU_segment

In [18]:
def detect_cell_thresh(image, seedcoord, thresh):
    # push to GPU + create a binary image
    image = cp.asarray(image)
    bin_img = image > thresh
    # floodfill the detected cell
    floodseed = tuple((seedcoord[0],seedcoord[1],seedcoord[2]))
    
    bin_img_cpu = bin_img.get()
    
    cellimg = skimagecpu.segmentation.flood(bin_img_cpu,floodseed)
    
    return np.array(cellimg).astype(bool)

In [19]:
# test the function
detect_cell_thresh(img, locmax[100], 900)

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [20]:
# define a function that adjusts the contrast until a target pixel number (volume) is reached
# in sams data (by hand) cell volume varies from ~300-3500 (more than 90% coverage)
# 1207um**3 +/- SD 803um**3 
# 1pixel has the volume

meanvol = int(1207/pixvol)
sdvol = int((800/pixvol))
print("tolerance 1 SD is from", meanvol-sdvol, "to", meanvol + sdvol, "voxels")
print("tolerance 0.5 SD is from", meanvol-0.5*sdvol, "to", meanvol + 0.5*sdvol, "voxels")
print("tolerance 2 SD is from", meanvol-2*sdvol, "to", meanvol + 2*sdvol, "voxels")

tolerance 1 SD is from 17940 to 88462 voxels
tolerance 0.5 SD is from 35570.5 to 70831.5 voxels
tolerance 2 SD is from -17321 to 123723 voxels


In [52]:
# iterative cell detection
def detect_cell_iter(image, seedcoord, expandpix, vlow, vhigh):
    
    # setup a threshold for the iterating, start with a little less than Otsu
    # this way it reduces the volume from a too large fit
    thresh_iter = find_cell_thresh(image, seedcoord, expandpix)*0.6
    void_mask = cp.zeros_like(image)
    
    # get the candidate cell mask
    CCM = detect_cell_thresh(image, seedcoord, thresh_iter)
    # count the number of pixels in the mask (volume)
    vol = cp.count_nonzero(cp.asarray(CCM))
    
    n_tries = 1
    # if the volume is within the tolerance, return the mask
    if (vol < vhigh and vol >vlow):
        print("done in one go, memory used: ", getGPUmem())
        return CCM
    
    # while the number of pixels is outside the tolerance
    while not(vol < vhigh and vol >vlow):
        # if the volume is larger than target interval set threshold to previous*1.x
        if vol > vhigh:
            #print("too large")
            thresh_iter = thresh_iter*1.2
            n_tries = n_tries + 1
            CCM = detect_cell_thresh(image, seedcoord, thresh_iter)
            #update volume
            vol = cp.count_nonzero(cp.asarray(CCM))
            
        # if the volume is below target interval set threshold to previous*0.x
        if vol < vlow:
            #print("too small")
            thresh_iter = thresh_iter*0.8
            n_tries = n_tries + 1
            CCM = detect_cell_thresh(image, seedcoord, thresh_iter)
            #update volume
            vol = cp.count_nonzero(cp.asarray(CCM))
            
        # if the number of iterations is high and the cellmask is tiny than an absolute minimum, break and return empty mask
        if (vol < vlow and n_tries > 10):
            #print("Bad seed: Just a specle, memory used: ", getGPUmem())
            return void_mask.get()
        
        # if the number of iterations is high and the cell mask is massive, the seed is on the bg, break and return empty mask
        if (vol > vhigh*3 and n_tries > 10):
            #print("Bad seed: bg pixel, memory used: ", getGPUmem())
            return void_mask.get()
        
        # if a reasonable volume is found return it
        if (vol < vhigh and vol >vlow):
            #print("Found mask in", n_tries, "iterations, memory used: ", getGPUmem())
            return CCM
        
        # if no solution is found (bouncing right between to high and too low)
        if (vol > vhigh and n_tries > 20):
            #print("Bad seed: Cant find a solution")
            return void_mask.get()
        if (vol < vlow and n_tries > 20):
            #print("Bad seed: Cant find a solution")
            return void_mask.get()

In [51]:
allcells_mask = np.zeros_like(img)

for i_cell in tqdm(range(locmax.shape[0])):
    cellmask = detect_cell_iter(img,                    
                     locmax[i_cell],
                     int(50//xystep),
                     17940,
                     123723)
        
    # update the complete mask
    allcells_mask =  allcells_mask + cellmask 

  0%|▏                                                                       | 1/555 [00:01<18:18,  1.98s/it]

done in one go, memory used:  0.7227937868933049


  0%|▎                                                                       | 2/555 [00:02<11:22,  1.23s/it]

done in one go, memory used:  0.7227937868933049


  1%|▍                                                                       | 3/555 [00:05<17:15,  1.88s/it]

done in one go, memory used:  0.7227937868933049


  1%|▌                                                                       | 4/555 [00:06<13:07,  1.43s/it]

done in one go, memory used:  0.7227937868933049
too large


  1%|▌                                                                       | 4/555 [00:09<21:22,  2.33s/it]


KeyboardInterrupt: 

In [23]:
#allcells_mask = allcells_mask.get()

In [24]:
viewer = napari.view_image(img)

new_layer = viewer.add_image(allcells_mask, opacity = 0.2, colormap = "magma")


NameError: name 'allcells_mask' is not defined

The loop above becomes slower the longer it runs, I assume because of memory issues.
Alternatively, just write the binary image to file.

In [ ]:
#create a directory for results
#os.mkdir("results")


In [26]:
for i_cell in tqdm(range(locmax.shape[0])):
    cellmask = detect_cell_iter(img,                    
                     locmax[i_cell],
                     int(50//xystep),
                     17940,
                     123723)
    
    # write mask to file
    iterfile = "./results/"+os.path.splitext(testimage)[0]+"_"+str(i_cell)+".tiff"
    skimagecpu.io.imsave(iterfile, cellmask.astype(int))
 

  0%|                                                                                | 0/555 [00:00<?, ?it/s]/tmp/ipykernel_15091/3174746727.py:10: UserWarning: ./results/C2-220421 otof_iba slide002 mouse195 005_0.tiff is a low contrast image
  skimagecpu.io.imsave(iterfile, cellmask.astype(int))
  0%|▏                                                                     | 1/555 [00:23<3:40:34, 23.89s/it]/tmp/ipykernel_15091/3174746727.py:10: UserWarning: ./results/C2-220421 otof_iba slide002 mouse195 005_1.tiff is a low contrast image
  skimagecpu.io.imsave(iterfile, cellmask.astype(int))
  0%|▎                                                                     | 2/555 [00:28<1:56:57, 12.69s/it]/tmp/ipykernel_15091/3174746727.py:10: UserWarning: ./results/C2-220421 otof_iba slide002 mouse195 005_2.tiff is a low contrast image
  skimagecpu.io.imsave(iterfile, cellmask.astype(int))
  1%|▍                                                                     | 3/555 [00:34<1:29:17,  9.71s/

KeyboardInterrupt: 